In [ ]:
from pybis import Openbis
import pandas as pd

We could do most of the operations directly using PyBis, but encapsulating operations is generally a good idea, and in a regular Python script, could be better used. A simple thing is having the OpenBis object ("o") part of the helper class, to prevent sharing it everywhere as we do here.
Aside the obvious common operations (such as connect, login, logout) it is a good idea to specialize the method for the need, i.e. fix the project name, use a specific type, prepare the data...

Connect, login and logout.

In [ ]:
def connect(url, username, password):
    o = Openbis(url)
    o.login(username, password, save_token=True)  # save the session token in ~/.pybis/example.com.token
    
    return o

def logout(o):
    o.logout()

def getToken(o):
    token = o.token()
        
def connectUsingPersonalAccessToken(o):
    self.pat = o.get_or_create_personal_access_token(sessionName="PETabViaAPI")
    o.set_token(self.pat, save_token=True)

Create and get a new experiment (or collection)
We assume that a space (place to share project) and a project already exists, but they can also be created in a similar way through pybis.

In [ ]:
def createExperiment(o, name):
    exp = o.new_experiment(
        code = name,
        type = 'DEFAULT_EXPERIMENT',
        project = 'PE_TABS'
    )

    exp.save()

    return exp

def createExperiment(o, name, new_type):
    exp = o.new_experiment(
        code = name,
        type = new_type,
        project = 'PE_TABS'
    )
    exp.set_props({'parameter_file': 'parameters_petab.tsv'})
    exp.set_props({'format_version': 1})
    exp.set_props({'condition_files': 'experimentalCondition_petab.tsv'})
    exp.set_props({'measurement_files': 'measurementData_petab.tsv'})
    exp.set_props({'sbml_files': 'model_petab.xml'})
    exp.set_props({'observable_files': 'observables_petab.tsv'})

    exp.save()

    return exp

def getExperiment(o, name):
    exp = o.get_experiment(name)

    return exp

Create an object (sample)

In [ ]:
# Objects are Samples
def createObject(o, name, new_type, for_experiment):
    obj = o.new_object(
        type = new_type,
        code = name,
        space='Openbisapiuser',
        experiment=for_experiment,
        props={"name": name, "description": name + " for PETAB"}
    )

    obj.save()

    return obj
    


Try to upload a spreadsheet (from a tsb)
Somehow could not make it works, and if the data is wrong, the full object cannot be seen correctly afterwards.
NB: headers seems to be fixed, A to Z, then AA, AB, ... , BA, ...

In [ ]:
# Does not work!
def setSpreadSheetFromFile(o, for_object, for_property_type):
    data = pd.read_csv("some_data.tsv",sep='\t')

    print(data)

    spreadsheet = {}

    # Headers seems to be fixed, A to Z, then AA, AB, ...
    spreadsheet["headers"] = ["A","B","C","D","E","F","G","H","I"]

    spreadsheet["data"] = data.iloc[1:].to_numpy().tolist()

    print(json.dumps(spreadsheet))
    #for_object.props["general_protocol.spreadsheet"] = "<DATA>" + str(
    #    base64.b64encode(bytes(json.dumps(spreadsheet), encoding='utf-8')), encoding='utf-8') + "</DATA>"

    for_object.set_props({for_property_type: "<DATA>" + str(
    base64.b64encode(bytes(json.dumps(spreadsheet), encoding='utf-8')), encoding='utf-8') + "</DATA>"})

It is possible to define a new type for each entity, but only as instance admin!
So it might be better to not do that through the API

Validation plugin are jython script that can validate the data

Type should have properties: data directly on the entity

In [ ]:
def createObjectType(o, name):
    objType = o.new_object_type(
        code = name,
        generatedCodePrefix='PETAB',
        validationPlugin=None
    )
    objType.save()

    return objType

def get_object_type(o, name):
    objType = o.get_object_type(name)
    return objType

In [ ]:
def createExperimentType(o, name):
    expType = o.new_experiment_type(
        code = name,
        description="PE Tab experiment type",
        validationPlugin=None
    )
    expType.save()

    return expType

def get_experiment_type(o, name):
    expType = o.get_experiment_type(name)
    return expType

Here we define some properties, starting with spreadsheet

Each property needs to be attached to an object.
The mandatory field is important-> if you are unsure which format you need to write, you better have it to false, as a wrong data in a property can break the object (will not show anything in the UI). It will need to be corrected in the classic UI or via API.

Create spread sheet visibly does not work: it creates a text field before the spreadsheet that cannot be filled at all.

The big method creating and assigning many properties it not a good idea, as commented, as each property can only be created and assigned once. So doing everything at the same time is not exactly safe.

In [ ]:
def createSpreadSheet(o, name, new_label, new_description):
    pt_spread = o.new_property_type(
        code="PET_TAB_"+name,
        label=new_label,
        description=new_description,
        dataType='XML',
        metaData={'custom_widget': 'Spreadsheet'}
    )
    pt_spread.save()

    return pt_spread

def assignProperty(o, to_what, property_name_to_assign, is_mandatory=True):
    # Should check that the to_what can be assigned a property
    to_what.assign_property(
        prop="PET_TAB_"+property_name_to_assign,  # mandatory
        section='',
        ordinal=1,
        mandatory=is_mandatory,
        initialValueForExistingEntities='initial value',
        showInEditView = True,
        showRawValueInForms = False
    )

# Actually not a good idea to do both creation and assignment in the same method->
# If a creation fails, the property is still there, creating it again will not work.
def createAndAttachPETabProperties(o, to_what):
    pt_text = o.new_property_type(
        code='parameter_file',
        label='parameter_file',
        description='parameter_file',
        dataType='VARCHAR',
    )
    pt_text.save()

    to_what.assign_property(
        prop='parameter_file',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='parameters_petab.tsv',
        showInEditView=True,
        showRawValueInForms=False
    )

    pt_int = o.new_property_type(
        code='format_version',
        label='format_version',
        description='format_version',
        dataType='INTEGER',
    )
    pt_int.save()

    to_what.assign_property(
        prop='format_version',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities=1,
        showInEditView=True,
        showRawValueInForms=False
    )

    pt_text = o.new_property_type(
        code='condition_files',
        label='condition_files',
        description='condition_files',
        dataType='VARCHAR',
    )
    pt_text.save()

    to_what.assign_property(
        prop='condition_files',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='experimentalCondition_petab.tsv',
        showInEditView=True,
        showRawValueInForms=False
    )

    pt_text = o.new_property_type(
        code='measurement_files',
        label='measurement_files',
        description='measurement_files',
        dataType='VARCHAR',
    )
    pt_text.save()

    to_what.assign_property(
        prop='measurement_files',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='measurementData_petab.tsv',
        showInEditView=True,
        showRawValueInForms=False
    )

    pt_text = o.new_property_type(
        code='sbml_files',
        label='sbml_files',
        description='sbml_files',
        dataType='VARCHAR',
    )
    pt_text.save()

    to_what.assign_property(
        prop='sbml_files',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='model_petab.xml',
        showInEditView=True,
        showRawValueInForms=False
    )

    pt_text = o.new_property_type(
        code='observable_files',
        label='observable_files',
        description='observable_files',
        dataType='VARCHAR',
    )
    pt_text.save()

    to_what.assign_property(
        prop='observable_files',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='observables_petab.tsv',
        showInEditView=True,
        showRawValueInForms=False
    )

def setDefaultPE_Tab_Values(o, for_experiment):
    for_experiment.set_props({'parameter_file': 'parameters_petab.tsv'})
    for_experiment.set_props({'format_version': 1})
    for_experiment.set_props({'condition_files': 'experimentalCondition_petab.tsv'})
    for_experiment.set_props({'measurement_files': 'measurementData_petab.tsv'})
    for_experiment.set_props({'sbml_files': 'model_petab.xml'})
    for_experiment.set_props({'observable_files': 'observables_petab.tsv'})

    for_experiment.save()

def createAndAttachNameAndDescProperties(o, to_what):
    #pt_text = o.new_property_type(
    #    code='name',
    #    label='name',
    #    description='name',
    #    dataType='VARCHAR',
    #)
    #pt_text.save()

    to_what.assign_property(
        prop='name',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='None',
        showInEditView=True,
        showRawValueInForms=False
    )

    to_what.assign_property(
        prop='description',  # mandatory
        section='',
        ordinal=1,
        mandatory=True,
        initialValueForExistingEntities='None',
        showInEditView=True,
        showRawValueInForms=False
    )

Creating dataset. Save upload them to OpenBis

In [ ]:
def createDataSet(o, for_file, for_object, for_exp):
    ds_new = o.new_dataset(
        type='ANALYZED_DATA',
        experiment=for_exp,
        sample=for_object,
        files=[for_file]
    )
    ds_new.save()

def createDataSetFromZip(o, for_file, for_object):
    ds_new = o.new_dataset(
        type='ANALYZED_DATA',
        sample=for_object,
        zipfile=for_file
    )
    ds_new.save()

Getting elements from OpenBIs

Experiments

In [ ]:
def getPETABsExperiment(o, name):
    exp = o.get_experiment("/OPENBISAPIUSER/PE_TABS/"+name)

    return exp

def getTestExperiments(o):
    # /OPENBISAPIUSER/TESTCONTENT/TESTCONTENT_EXP_1
    experiments = o.get_experiments(
        project='TESTCONTENT',
        space='OPENBISAPIUSER'
    )

    #print("Found "+experiments.count.str()+" experiments")

    for experiment in experiments:  # iterate over search results
        print(experiment.props.all())

    return experiments

def getPETaBsExperiments(o):
    experiments = o.get_experiments(
        project='PE_TABS',
        space='OPENBISAPIUSER',
        type='PE_Tab_Test_Type',
        tags='*',
        finished_flag=False,
        props=['name', 'finished_flag']
    )

    #print("Found "+experiments.count.str()+" experiments")

    for experiment in experiments:  # iterate over search results
        print(experiment.props.all())

    return experiments

def getExperiments(o, for_project):
    # /OPENBISAPIUSER/TESTCONTENT/TESTCONTENT_EXP_1
    experiments = for_project.get_experiments()

    #print("Found "+experiments.count.str()+" experiments")

    for experiment in experiments:  # iterate over search results
        print(experiment.props.all())

    return experiments

def getExperiment(o, name, for_project):
    exp = for_project.get_experiment(code=name)

    return exp

def getExperimentByFullPath(o, path):
    exp = o.get_experiment(path)

    return exp

Project

In [ ]:
def getProject(o, name):
    pro = o.get_project("/OPENBISAPIUSER/"+name)

    return pro

Samples

In [ ]:
def getSamples(o, for_experiment):
    samples = for_experiment.get_samples()

    return samples

In [ ]:
def getDataset(o, id):
    dataset = o.get_dataset(id)

    return dataset

def getAllDatasetInCollection(o, forCollection):
    datasets = forCollection.get_datasets()

    return datasets

Running everything:

First, commented out, is the writting part. You generally can/should only do it once!
Then come the reading. The user for reading might be a read-only user. 

In [ ]:
o = connect("https://youropenbis-server.org", "oneAPIUser", "password")

# Longer validity:
# connectUsingPersonalAccessToken(o)

print(f"Session is active: {o.is_session_active()} and token is {o.token}")

# Only once
# Need to be instance admin! So might not be practical (or secured) as API call
#createSpreadSheet(o, "measurementData", "PETab_MeasurementData", "Data for measurement")

# Only once! Need to be instance admin! So might not be practical (or secured) as API call
#petTabExperimentType = createExperimentType(o, "PE_Tab_Test_Type")
#petTabExperimentType = get_experiment_type(o, "PE_Tab_Test_Type")

#petTabObjectType = createObjectType(o, "PE_Tab_Object_Test_Type")
#petTabObjectType = get_object_type(o, "PE_Tab_Object_Test_Type")

#assignProperty(o, petTabObjectType, "measurementData")

#createAndAttachNameAndDescProperties(o, petTabObjectType)
# Only once
#createAndAttachPETabProperties(o, petTabExperimentType)

#onePeTabExperiment = createExperiment(o, "PE_Tab_Test_5", "PE_Tab_Test_Type")
#onePeTabExperiment = getExperiment(o, "/OPENBISAPIUSER/PE_TABS/PE_TAB_TEST_3")

#onePeTabObject = createObject(o, "PE_Tab_Test_Object_2", "PE_Tab_Object_Test_Type", onePeTabExperiment)
#setDefaultPE_Tab_Values(o, onePeTabExperiment)
#setSpreadSheetFromFile(o, onePeTabObject, 'pet_tab_measurementdata')

#createDataSet(o, "some_data.tsv", "/OPENBISAPIUSER/PE_TABS/PE_TAB_TEST_3/PE_Tab_Test_Object_2", "/OPENBISAPIUSER/PE_TABS/PE_TAB_TEST_3")

#createDataSetFromZip(o,
#    "example.zip",
#    "/OPENBISAPIUSER/PE_TABS/PE_TAB_TEST_3/PE_Tab_Test_Object_2")

getTestExperiments(o)
getPETaBsExperiments(o)

oneExperiment = getPETABsExperiment(o, "PE_TAB_TEST_3")
print("PE_TAB_TEST_3 Experiment")
print(oneExperiment.props.all())

oneProject = getProject(o, "PE_TABS")
print("All Experiments in PE_TABS project")
allPE_TABS_experiments = getExperiments(o, oneProject)

for anExperiment in allPE_TABS_experiments:
    print(anExperiment.props.all())
    allSamples = getSamples(o, anExperiment)

    for aSample in allSamples:  # iterate over search results
        print(aSample.props.all())

oneProject = getProject(o, "TESTCONTENT")
print("All Experiments in TESTCONTENT project")
getExperiments(o, oneProject)

print("EXP5 Experiment in TESTCONTENT project")
anotherExperiment = getExperimentByFullPath(o, "/OPENBISAPIUSER/TESTCONTENT/TESTCONTENT_EXP_1")
print(anotherExperiment.props.all())

print("All samples in EXP5 Experiment")
allSamples = getSamples(o, anotherExperiment)

for aSample in allSamples:  # iterate over search results
    print(aSample.props.all())

print("TESTCOLLECTION1 Experiment in TESTCONTENT project, containing our dataset")
collExperiment = getExperimentByFullPath(o, "/OPENBISAPIUSER/TESTCONTENT/TESTCOLLECTION1")
print(collExperiment.props.all())
allDataSetsInColl = getAllDatasetInCollection(o, collExperiment)

print("Datasets in TESTCOLLECTION1 Experiment")
for aDataset in allDataSetsInColl:  # iterate over search results
    print(aDataset.props())
    print(aDataset.file_list)



Dataset ID can be found in OpenBis

In [ ]:
dataset = getDataset("20221227143950020-29")
print(dataset.props())
print(dataset.file_list)
dataset.download()

logout()